# Targeted childcare entitlement

[Official government guidance](https://www.gov.uk/help-with-childcare-costs/free-childcare-2-year-olds) | [Legislation](https://www.legislation.gov.uk/uksi/2014/2147/made)

## Program description

### Overview

The targeted childcare entitlement (also known as the **2-year-old offer**) provides **15 hours** of free childcare per week for eligible 2-year-old children. The entitlement totals **570 hours** annually and it delivered across **38 weeks**. This program specifically targets families who meet certain income or benefit criteria.

### Eligibility requirements

#### Age criteria
Children must be **exactly 2 years old** - not younger or older. Once a child turns 3, they transition to the universal or extended childcare entitlement programs.

#### Benefit-based eligibility
Families qualify if they receive any of these benefits:
* **Income Support**
* **Income-based Jobseeker's Allowance (JSA)**
* **Income-related Employment and Support Allowance (ESA)**
* **Pension Credit (Guarantee Credit)**

#### Eligibility conditions
Families may also qualify based on income thresholds if they receive:
* **Universal Credit** with household earned income of **£15,400 or less** per year
* **Tax Credits** (Working Tax Credit or Child Tax Credit) with household income of **£16,190 or less** per year

#### Geographic scope
The program applies only to families living in **England**. Scotland, Wales, and Northern Ireland administer their own early education systems.

## Implementation

The targeted childcare entitlement program is implemented through several interconnected components:

### 1. Parameter definition
Located in [`parameters/gov/dfe/targeted_childcare_entitlement/`](https://github.com/PolicyEngine/policyengine-uk/tree/master/policyengine_uk/parameters/gov/dfe/targeted_childcare_entitlement), key parameters include:

* Program eligibility:
  * [`age_eligibility.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/parameters/gov/dfe/targeted_childcare_entitlement/age_eligibility.yaml): Defines the eligible age (2 years old)
  * [`qualifying_benefits.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/parameters/gov/dfe/targeted_childcare_entitlement/qualifying_benefits.yaml): Lists qualifying benefits
  * [`qualifying_criteria.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/parameters/gov/dfe/targeted_childcare_entitlement/qualifying_criteria.yaml): Specifies additional eligibility conditions

* Income thresholds:
  * [`income_limit/tax_credits.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/parameters/gov/dfe/targeted_childcare_entitlement/income_limit/tax_credits.yaml): Sets £16,190 threshold for Tax Credit recipients
  * [`income_limit/universal_credit.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/parameters/gov/dfe/targeted_childcare_entitlement/income_limit/universal_credit.yaml): Sets £15,400 threshold for Universal Credit recipients

* Entitlement specifics:
  * [`hours_entitlement.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/parameters/gov/dfe/targeted_childcare_entitlement/hours_entitlement.yaml): Defines 570 annual entitlement hours
  * [`childcare_funding_rate.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/parameters/gov/dfe/childcare_funding_rate.yaml): Sets funding rates by age

### 2. Benefit-based eligibility assessment
Managed in [`targeted_childcare_entitlement_eligible.py`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/variables/gov/dfe/targeted_childcare_entitlement/targeted_childcare_entitlement_eligible.py):
* Checks if family resides in England
* Verifies receipt of qualifying benefits defined in `qualifying_benefits.yaml`
* References additional qualifying criteria for means-tested benefits

Example table:

| Benefit type | Receiving benefit | Meets criteria | Is eligible |
|--------------|-------------------|----------------|-------------|
| Income Support | Yes              | N/A            | Yes         |
| JSA (income-based) | Yes           | N/A            | Yes         |
| ESA (income-related) | Yes         | N/A            | Yes         |
| Pension Credit (Guarantee) | Yes   | N/A            | Yes         |
| Universal Credit | Yes            | No             | No          |
| Tax Credits      | Yes            | No             | No          |
| None             | No             | N/A            | No          |

### 3. Benefit eligibility
Benefit eligibility is implemented in separate files for different benefit programs:

#### Universal Credit criteria
Universal Credit criteria is handled by [`meets_universal_credit_criteria_for_targeted_childcare_entitlement.py`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/variables/gov/dfe/targeted_childcare_entitlement/meets_universal_credit_criteria_for_targeted_childcare_entitlement.py):
* Verifies UC receipt by checking if benefit amount is greater than zero
* Compares earned income against £15,400 threshold
* Returns boolean eligibility determination

Example table:

| Universal Credit amount | Earned income | Below £15,400 threshold | Meets UC criteria |
|------------------------|---------------|------------------------|-------------------|
| £500                   | £14,000       | Yes                    | Yes               |
| £300                   | £16,000       | No                     | No                |
| £0                     | £10,000       | Yes                    | No                |

#### Tax Credits criteria
Managed by [`meets_tax_credit_criteria_for_targeted_childcare_entitlement.py`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/variables/gov/dfe/targeted_childcare_entitlement/meets_tax_credit_criteria_for_targeted_childcare_entitlement.py):
* Checks for either Child Tax Credit or Working Tax Credit receipt
* Compares applicable income against £16,190 threshold
* Returns boolean eligibility determination

Example table:

| Tax Credit type | Credit amount | Applicable income | Below £16,190 threshold | Meets TC criteria |
|-----------------|---------------|-------------------|------------------------|-------------------|
| Child Tax Credit | £2,000        | £15,000           | Yes                    | Yes               |
| Working Tax Credit | £1,000      | £16,500           | No                     | No                |
| Both             | £3,000        | £14,000           | Yes                    | Yes               |
| None             | £0            | £10,000           | Yes                    | No                |

### 4. Entitlement calculation
Final calculation in [`targeted_childcare_entitlement.py`](https://github.com/PolicyEngine/policyengine-uk/blob/master/policyengine_uk/variables/gov/dfe/targeted_childcare_entitlement/targeted_childcare_entitlement.py):
* Checks if child's age is exactly 2 years using `age_eligibility` parameter
* Applies entitlement hours (570 annually) for eligible children
* Multiplies by appropriate funding rate based on age (£8.28 per hour for 2-year-olds in 2024)
* Returns final annual entitlement amount

Example table:

| Child age | Eligible | Hours entitlement | Hourly rate | Annual entitlement |
|-----------|----------|-------------------|-------------|--------------------|
| 1         | No       | 0                 | £11.22      | £0                 |
| 2         | Yes      | 570               | £8.28       | £4,719.60          |
| 3         | No       | 0                 | £5.88       | £0                 |
| 4         | No       | 0                 | £5.88       | £0                 |